<a href="https://colab.research.google.com/github/ecaidav/OpenSecrets-Donor-Lookup-Scraper/blob/main/OpenSecrets-Donor-Lookup-Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium webdriver-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y wget unzip
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'
!apt-get update
!apt-get install -y google-chrome-stable

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,088 kB]
Get:13 http://se

In [3]:
!wget https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.141/linux64/chromedriver-linux64.zip
!unzip chromedriver-linux64.zip -d /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver-linux64/chromedriver

--2024-06-08 00:33:25--  https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.141/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.27, 172.217.12.27, 142.250.65.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8713889 (8.3M) [application/zip]
Saving to: ‘chromedriver-linux64.zip’

chromedriver-linux6 100%[===================>]   8.31M  --.-KB/s    in 0.06s   

2024-06-08 00:33:25 (139 MB/s) - ‘chromedriver-linux64.zip’ saved [8713889/8713889]

Archive:  chromedriver-linux64.zip
  inflating: /usr/local/bin/chromedriver-linux64/LICENSE.chromedriver  
  inflating: /usr/local/bin/chromedriver-linux64/chromedriver  


In [4]:
!ls /usr/local/bin/chromedriver-linux64/chromedriver

/usr/local/bin/chromedriver-linux64/chromedriver


In [5]:
import pandas as pd
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

pd.set_option('display.max_columns', None)

In [8]:
# load csv file of cleaned dataset for top 15 wealthiest New Yorkers, according to Forbes (dated April 2023) as the dataframe donor_data

donor_data = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTsbacX1zw34YzjO5KMA0idGWgRSIwlBh98_WaU7TQOhUPnOnbIg1mPyuXllfH0JnOAl8X-W92Yk0QD/pub?gid=0&single=true&output=csv")
donor_data['FULL NAME'] = donor_data['FULL NAME'].str.upper()
donor_data['FNAME'] = donor_data['FNAME'].str.upper()
donor_data['LNAME'] = donor_data['LNAME'].str.upper()

donor_data

,FNAME,LNAME,FULL NAME,RANK,AGE,WORTH,YEAR,MONTH,CATEGORY,SOURCE,COUNTRY,STATE,CITY,COUNTRY.1,ORGANIZATION,SELF_MADE,GENDER,BIRTH_DATE,TITLE,PHILANTHROPY,SIBLING_NUMBER,BIO,ABOUT
0,MICHAEL,BLOOMBERG,MICHAEL BLOOMBERG,7,81,94500,2023,4,Media & Entertainment,Bloomberg LP,United States,New York,New York,United States,Bloomberg,True,M,1942-02-14 0:00:00,CEO,4.0,NaN,Michael Bloomberg cofounded financial informat...,"In 2018, Bloomberg pledged $1.8 billion to Joh..."
1,JULIA,KOCH,JULIA KOCH,17,60,59000,2023,4,Diversified,Koch Industries,United States,New York,New York,United States,NaN,False,F,1962-04-12 0:00:00,NaN,2.0,NaN,Julia Koch and her three children inherited a ...,NaN
2,JIM,SIMONS,JIM SIMONS,49,84,28100,2023,4,Finance & Investments,Hedge funds,United States,New York,East Setauket,United States,Renaissance Technologies Corp.,True,M,1938-04-25 0:00:00,Founder,4.0,NaN,Jim Simons is the founder of Renaissance Techn...,Simons used to chair the math department at St...
3,STEPHEN,SCHWARZMAN,STEPHEN SCHWARZMAN,50,76,27800,2023,4,Finance & Investments,Investments,United States,New York,New York,United States,Blackstone Group,True,M,1947-02-14 0:00:00,Chairman and CEO,2.0,2.0,"The son of a dry goods store owner, Stephen Sc...",Blackstone is a combination of its founders' n...
4,LEONARD,LAUDER,LEONARD LAUDER,77,90,21000,2023,4,Fashion & Retail,Estee Lauder,United States,New York,New York,United States,The Estée Lauder Companies,False,M,1933-03-19 0:00:00,Chairman Emeritus,3.0,NaN,Leonard Lauder spent three decades running his...,Lauder gifted his massive Cubist art collectio...
5,RUPERT,MURDOCH,RUPERT MURDOCH,99,92,17100,2023,4,Media & Entertainment,"Newspapers, TV network",United States,New York,New York,United States,News Corp Class A,False,M,1931-03-11 0:00:00,Chairman and CEO,1.0,NaN,Murdoch controls a media empire that includes ...,"Murdoch bought The New York Post, his first ma..."
6,STEPHEN,ROSS,STEPHEN ROSS,147,82,11600,2023,4,Real Estate,Real estate,United States,New York,New York,United States,NaN,True,M,1940-05-10 0:00:00,NaN,2.0,NaN,Stephen Ross was a tax attorney before foundin...,The single largest donor to alma mater Univers...
7,ISRAEL,ENGLANDER,ISRAEL ENGLANDER,153,74,11300,2023,4,Finance & Investments,Hedge funds,United States,New York,New York,United States,"Millennium Management, L.L.C.",True,M,1948-09-30 0:00:00,Founder,1.0,NaN,Israel Englander founded Millennium Management...,NaN
8,DONALD,NEWHOUSE,DONALD NEWHOUSE,164,93,10700,2023,4,Media & Entertainment,Media,United States,New York,New York,United States,NaN,False,M,1929-08-05 0:00:00,NaN,2.0,NaN,"Donald Newhouse and his late brother, Samuel ""...","In 2016, 67 years after he left Syracuse U. to..."
9,VALERIE,MARS,VALERIE MARS,195,64,9600,2023,4,Food & Beverage,"Candy, pet food",United States,New York,New York,United States,NaN,False,F,1959-01-26 0:00:00,NaN,NaN,NaN,Valerie Mars and her three sisters each inheri...,NaN


In [7]:
# scrape OpenSecrets donor database via its Donor Lookup search engine using names from the Forbes csv file

def custom_title_case(s):
    return s.upper()

def convert_date(d):
    if pd.isna(d):
        return ""
    dateTimeObj = datetime.strptime(str(d), "%m-%d-%Y")
    return dateTimeObj.strftime("%Y-%m-%d")

def fix_recipient_name(recipient):
    recipient = re.sub(r'\(.*\)$', '', recipient).strip()
    parts = recipient.split(', ')
    if len(parts) > 1:
        first_part = custom_title_case(parts[0])
        rest = ' '.join(parts[1:])
        recipient_fixed = f"{custom_title_case(rest)} {first_part}"
    else:
        recipient_fixed = custom_title_case(recipient)
    return recipient_fixed

def split_contributor(contributor):
    p = contributor.split('  ')
    zip_code_match = re.search(r'(\b\d{5}\b)$', p[1])
    zip_code = zip_code_match.group(1) if zip_code_match else None

    village_match = re.search(r'(^[a-zA-Z ]+),', p[1])
    village = village_match.group(1) if village_match else None

    name_parts = p[0].split(', ')
    first_name = custom_title_case(' '.join(name_parts[1:])) if len(name_parts) > 1 else ''
    last_name = custom_title_case(name_parts[0])

    return last_name, first_name, village, zip_code

def clean_amount(amount):
    cleaned_amount = re.sub(r'[^\d]', '', amount)
    return int(cleaned_amount) if cleaned_amount else 0

def is_ny_zip_code(zip_code):
    return 10001 <= int(zip_code) <= 14975 if zip_code and zip_code.isdigit() else False

def process_data(url):
    disclaimer = 'FEDERAL LAW PROHIBITS THE USE OF CONTRIBUTOR INFORMATION FOR THE PURPOSE OF SOLICITING CONTRIBUTIONS OR FOR ANY COMMERCIAL PURPOSE.'
    all_data = []
    page = 1

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    service = Service('/usr/local/bin/chromedriver-linux64/chromedriver')

    driver = webdriver.Chrome(service=service, options=chrome_options)

    while True:
        full_url = f"{url}&page={page}"
        driver.get(full_url)

        try:
            table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//table'))
            )

            table_html = table.get_attribute('outerHTML')

            df = pd.read_html(table_html)[0]

            if df.empty:
                break
            else:
                all_data.append(df)
                page += 1

        except Exception as e:
            break

    driver.quit()

    if all_data:
        df = pd.concat(all_data, ignore_index=True)
        df.drop(df.tail(1).index, inplace=True)
        df.columns = ['Category', 'Contributor', 'Employer', 'Occupation', 'Date', 'Amount', 'Recipient', 'Recipient Jurisdiction']
        df['Party'] = df['Recipient'].str.extract(r'\(([A-Z]+)\)$')
        df['Recipient'] = df['Recipient'].apply(fix_recipient_name)
        df['Amount'] = df['Amount'].apply(clean_amount)
        df = df[~df['Category'].str.contains(disclaimer)]
        split_contributions = df['Contributor'].apply(lambda x: pd.Series(split_contributor(x), index=['Last Name', 'First Name', 'Village', 'Zip Code']))
        df = pd.concat([df, split_contributions], axis=1)
        df = df[df['Zip Code'].apply(is_ny_zip_code)]
        df['Date'] = df['Date'].apply(convert_date)

        columns_order = ['Category', 'Contributor', 'Last Name', 'First Name', 'Village', 'Zip Code', 'Employer', 'Occupation', 'Date', 'Amount', 'Recipient', 'Party', 'Recipient Jurisdiction']
        df = df[columns_order]
    else:
        df = pd.DataFrame()

    return df

def generate_urls_and_aggregate_data(donor_data):
    base_url = "https://www.opensecrets.org/donor-lookup/results?name="
    aggregated_data = pd.DataFrame()
    aggregated_data['Donor Full Name'] = None

    for index, row in donor_data.iterrows():
        full_name = f"{row['FNAME']} {row['LNAME']}"
        url = f"{base_url}{full_name}"
        data = process_data(url)
        if not data.empty:
            data['Donor Full Name'] = full_name
            aggregated_data = pd.concat([aggregated_data, data], ignore_index=True)
    return aggregated_data

aggregated_data = generate_urls_and_aggregate_data(donor_data)
aggregated_data

,Donor Full Name,Category,Contributor,Last Name,First Name,Village,Zip Code,Employer,Occupation,Date,Amount,Recipient,Party,Recipient Jurisdiction
0,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NEW YORK, 10075",BLOOMBERG,MICHAEL,NEW YORK,10075,"BLOOMBERG, LP",OWNER,2014-03-27,5200.0,BRIAN SCHATZ,D,Federal
1,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NY, 10022",BLOOMBERG,MICHAEL,NY,10022,BLOOMBERG FINANCIAL,NaN,2000-03-02,1000.0,MICHAEL G OXLEY,R,Federal
2,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NEW YORK, 10022",BLOOMBERG,MICHAEL,NEW YORK,10022,BLOOMBERG LP,NaN,1995-04-19,1000.0,CHARLES E SCHUMER,D,Federal
3,MICHAEL BLOOMBERG,Money to SuperPAC/Outside Group,"BLOOMBERG, MICHAEL NEW YORK, 10075",BLOOMBERG,MICHAEL,NEW YORK,10075,BLOOMBERG LP,EXECUTIVE,2018-08-20,463125.0,WOMEN VOTE!,NaN,Federal
4,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NEW YORK, 10022",BLOOMBERG,MICHAEL,NEW YORK,10022,Bloomberg L P,NaN,1996-03-26,1000.0,CHARLES E SCHUMER,D,Federal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2693,LEONARD STERN,Money to PACs,"STERN, LEONARD N MR NEW YORK, 10065",STERN,LEONARD N MR,NEW YORK,10065,THE HARTZ GROUP INC.,CEO,2023-06-21,100000.0,SFA FUND,NaN,Federal
2694,LEONARD STERN,Money to PACs,"STERN, LEONARD N MR NEW YORK, 10021",STERN,LEONARD N MR,NEW YORK,10021,THE HARTZ GROUP INC,CEO,2024-01-22,5000.0,STAND FOR AMERICA PAC,R,Federal
2695,LEONARD STERN,Money to PACs,"STERN, LEONARD N MR NEW YORK, 10065",STERN,LEONARD N MR,NEW YORK,10065,THE HARTZ GROUP INC.,CEO,2023-03-13,5000.0,SFA FUND,NaN,Federal
2696,LEONARD STERN,Money to PACs,"STERN, LEONARD N MR NEW YORK, 10021",STERN,LEONARD N MR,NEW YORK,10021,THE HARTZ GROUP INC.,CEO,2023-03-13,5000.0,STAND FOR AMERICA PAC,R,Federal


In [10]:
# merge the scraped data (dataframe aggregated_data) and merge with donor_data to maximize analytical capabilities

ny_donors = aggregated_data.merge(donor_data, left_on='Donor Full Name', right_on='FULL NAME')
ny_donors

,Donor Full Name,Category,Contributor,Last Name,First Name,Village,Zip Code,Employer,Occupation,Date,Amount,Recipient,Party,Recipient Jurisdiction,FNAME,LNAME,FULL NAME,RANK,AGE,WORTH,YEAR,MONTH,CATEGORY,SOURCE,COUNTRY,STATE,CITY,COUNTRY.1,ORGANIZATION,SELF_MADE,GENDER,BIRTH_DATE,TITLE,PHILANTHROPY,SIBLING_NUMBER,BIO,ABOUT
0,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NEW YORK, 10075",BLOOMBERG,MICHAEL,NEW YORK,10075,"BLOOMBERG, LP",OWNER,2014-03-27,5200.0,BRIAN SCHATZ,D,Federal,MICHAEL,BLOOMBERG,MICHAEL BLOOMBERG,7,81,94500,2023,4,Media & Entertainment,Bloomberg LP,United States,New York,New York,United States,Bloomberg,True,M,1942-02-14 0:00:00,CEO,4.0,NaN,Michael Bloomberg cofounded financial informat...,"In 2018, Bloomberg pledged $1.8 billion to Joh..."
1,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NY, 10022",BLOOMBERG,MICHAEL,NY,10022,BLOOMBERG FINANCIAL,NaN,2000-03-02,1000.0,MICHAEL G OXLEY,R,Federal,MICHAEL,BLOOMBERG,MICHAEL BLOOMBERG,7,81,94500,2023,4,Media & Entertainment,Bloomberg LP,United States,New York,New York,United States,Bloomberg,True,M,1942-02-14 0:00:00,CEO,4.0,NaN,Michael Bloomberg cofounded financial informat...,"In 2018, Bloomberg pledged $1.8 billion to Joh..."
2,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NEW YORK, 10022",BLOOMBERG,MICHAEL,NEW YORK,10022,BLOOMBERG LP,NaN,1995-04-19,1000.0,CHARLES E SCHUMER,D,Federal,MICHAEL,BLOOMBERG,MICHAEL BLOOMBERG,7,81,94500,2023,4,Media & Entertainment,Bloomberg LP,United States,New York,New York,United States,Bloomberg,True,M,1942-02-14 0:00:00,CEO,4.0,NaN,Michael Bloomberg cofounded financial informat...,"In 2018, Bloomberg pledged $1.8 billion to Joh..."
3,MICHAEL BLOOMBERG,Money to SuperPAC/Outside Group,"BLOOMBERG, MICHAEL NEW YORK, 10075",BLOOMBERG,MICHAEL,NEW YORK,10075,BLOOMBERG LP,EXECUTIVE,2018-08-20,463125.0,WOMEN VOTE!,NaN,Federal,MICHAEL,BLOOMBERG,MICHAEL BLOOMBERG,7,81,94500,2023,4,Media & Entertainment,Bloomberg LP,United States,New York,New York,United States,Bloomberg,True,M,1942-02-14 0:00:00,CEO,4.0,NaN,Michael Bloomberg cofounded financial informat...,"In 2018, Bloomberg pledged $1.8 billion to Joh..."
4,MICHAEL BLOOMBERG,Money to Candidates,"BLOOMBERG, MICHAEL NEW YORK, 10022",BLOOMBERG,MICHAEL,NEW YORK,10022,Bloomberg L P,NaN,1996-03-26,1000.0,CHARLES E SCHUMER,D,Federal,MICHAEL,BLOOMBERG,MICHAEL BLOOMBERG,7,81,94500,2023,4,Media & Entertainment,Bloomberg LP,United States,New York,New York,United States,Bloomberg,True,M,1942-02-14 0:00:00,CEO,4.0,NaN,Michael Bloomberg cofounded financial informat...,"In 2018, Bloomberg pledged $1.8 billion to Joh..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2693,LEONARD STERN,Money to PACs,"STERN, LEONARD N MR NEW YORK, 10065",STERN,LEONARD N MR,NEW YORK,10065,THE HARTZ GROUP INC.,CEO,2023-06-21,100000.0,SFA FUND,NaN,Federal,LEONARD,STERN,LEONARD STERN,290,85,7600,2023,4,Real Estate,Real estate,United States,New York,New York,United States,"Hartz Mountain Industries, Inc.",False,M,1938-03-28 0:00:00,Chairman and CEO,1.0,NaN,Leonard Stern joined his father Max's pet supp...,Leonard's father Max Stern arrived in New York...
2694,LEONARD STERN,Money to PACs,"STERN, LEONARD N MR NEW YORK, 10021",STERN,LEONARD N MR,NEW YORK,10021,THE HARTZ GROUP INC,CEO,2024-01-22,5000.0,STAND FOR AMERICA PAC,R,Federal,LEONARD,STERN,LEONARD STERN,290,85,7600,2023,4,Real Estate,Real estate,United States,New York,New York,United States,"Hartz Mountain Industries, Inc.",False,M,1938-03-28 0:00:00,Chairman and CEO,1.0,NaN,Leonard Stern joined his father Max's pet supp...,Leonard's father Max Stern arrived in New York...
2695,LEONARD STERN,Money to PACs,"STERN, LEONARD N MR NEW YORK, 10065",STERN,LEONARD N MR,NEW YORK,10065,THE HARTZ GROUP INC.,CEO,2023-03-13,5000.0,SFA FUND,NaN,Federal,LEONARD,STERN,LEONARD STERN,290,85,7600,2023,4,Real Estate,Real estate,United States,New York,New 

In [11]:
# show total amount donated by donor

donors = ny_donors.groupby('Donor Full Name', as_index=False)['Amount'].sum()
donors.sort_values(by=['Amount'], ascending=False)

,Donor Full Name,Amount
8,MICHAEL BLOOMBERG,160413290.0
11,STEPHEN SCHWARZMAN,11085873.0
9,RUPERT MURDOCH,6690664.0
5,LEON BLACK,2200859.0
6,LEONARD LAUDER,1804171.0
10,STEPHEN ROSS,500886.0
1,DAVID SHAW,432643.0
3,JIM SIMONS,362950.0
7,LEONARD STERN,340368.0
4,JULIA KOCH,243805.0


In [12]:
# calculate ratio of amount donated by self-made vs. not self-made donor

made = ny_donors[ny_donors['SELF_MADE'] == True]
donors_made = made.groupby('Donor Full Name', as_index=False)['Amount'].sum()
donors_made.sort_values(by=['Amount'], ascending=False)

not_made = ny_donors[ny_donors['SELF_MADE'] == False]
donors_not = not_made.groupby('Donor Full Name', as_index=False)['Amount'].sum()
donors_not.sort_values(by=['Amount'], ascending=False)

made_v_not = donors_made['Amount'].sum() / donors_not['Amount'].sum()
made_v_not

19.283358490266778

In [13]:
# show amount donated by party

parties = ny_donors.groupby('Party', as_index=False)['Amount'].sum()
parties.sort_values(by=['Amount'], ascending=False)

,Party,Amount
0,D,7581884.0
3,R,5687991.0
1,I,47000.0
2,N,14600.0


In [14]:
# show amount received by recipient

recipients = ny_donors.groupby('Recipient', as_index=False)['Amount'].sum()
recipients.sort_values(by=['Amount'], ascending=False)

,Recipient,Amount
394,INDEPENDENCE USA PAC,62137659.0
388,HOUSE MAJORITY PAC,25661025.0
552,LCV VICTORY FUND,15600000.0
996,WOMEN VOTE!,12863125.0
873,SENATE LEADERSHIP FUND,10000000.0
...,...,...
189,COREY JOHNSON,50.0
438,JESSICA RAMOS,15.0
62,AUSTIN FRERICK,10.0
775,QUINN FOR NEW YORK,10.0


In [15]:
# amount donated to jurisdiction (federal and state)

juris = ny_donors.groupby('Recipient Jurisdiction', as_index=False)['Amount'].sum()
juris.sort_values(by=['Amount'], ascending=False)

,Recipient Jurisdiction,Amount
5,Federal,168351930.0
21,NV,3530000.0
22,NY,3427676.0
1,CA,2557600.0
24,OR,1095000.0
2,CO,1010325.0
25,PA,762000.0
13,MD,527833.0
6,GA,504700.0
8,IA,500010.0


In [16]:
# amount donated by party and jurisdiction (federal, NY and 2024 battleground states only)

ny_donors_filtered = ny_donors[ny_donors['Recipient Jurisdiction'].isin(['Federal', 'NY', 'NV', 'PA', 'GA', 'AZ', 'MI', 'WI'])]
part_juris = ny_donors_filtered.groupby(['Party', 'Recipient Jurisdiction'])['Amount'].sum().unstack(fill_value=0)
part_juris.reset_index(inplace=True)
part_juris.columns.name = None
part_juris = part_juris[['Party', 'Federal', 'NY', 'NV', 'PA', 'GA', 'AZ', 'MI', 'WI']]
part_juris

,Party,Federal,NY,NV,PA,GA,AZ,MI,WI
0,D,3767249.0,613700.0,30000.0,762000.0,504700.0,13000.0,8400.0,10000.0
1,I,47000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,R,2820880.0,2775611.0,0.0,0.0,0.0,0.0,6800.0,0.0
